In [7]:
import os
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torchvision.utils import save_image
import torch
import numpy as np
import matplotlib.pyplot as plt
from time import time
from torchsummary import summary

# Hyperparameters
number_epochs = 20
batch_size = 128
learning_rate = 1e-4

# Transformations
transform_image = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

# Dataset and DataLoader
dataset = datasets.MNIST('./data', download=True, train=True, transform=transform_image)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Display one batch shape
dataiter = iter(data_loader)
images, labels = next(dataiter)
print(images.shape)
print(labels.shape)

# Define the Autoencoder model
class AutoencoderModel(nn.Module):
    def __init__(self):
        super(AutoencoderModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True),
            nn.Linear(64, 12),
            nn.ReLU(True),
            nn.Linear(12, 3)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True),
            nn.Linear(128, 28 * 28),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Instantiate the model
model = AutoencoderModel()

# Model summary
summary(model, input_size=(1, 28 * 28))

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

# Helper function to convert tensor to image
def to_image(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

# Training loop
for epoch in range(number_epochs):
    running_loss = 0.0
    for data in data_loader:
        image, _ = data
        image = image.view(image.shape[0], -1)
        image = Variable(image)

        # Forward pass
        output = model(image)
        loss = criterion(output, image)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print loss for the epoch
    print(f"Epoch {epoch + 1}/{number_epochs} - Training loss: {running_loss/len(data_loader):.4f}")

    # Save images every 10 epochs
    if (epoch + 1) % 10 == 0:
        pic = to_image(output.cpu().data)
        os.makedirs('output', exist_ok=True)
        save_image(pic, f'output/image_{epoch + 1}.png')

# Save the trained model
torch.save(model.state_dict(), './sim_autoencoder.pth')

# Display the last image
plt.imshow(pic[pic.shape[0]-1].numpy().squeeze(), cmap='gray_r')
plt.show()


Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>



RuntimeError: Error downloading train-images-idx3-ubyte.gz

In [ ]:
import os
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
from torchvision import datasets, transforms
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import torch.nn.functional as F
number_epochs = 10
batch_size = 128
learning_rate = 1e-4

transform_image = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
dataset = datasets.MNIST('./data', download=True, train=True, transform=transform_image)
data_loader = DataLoader(dataset, batch_size=batch_size,shuffle=True)


dataiter = iter(data_loader)
images, labels = next(dataiter)
print(images.shape)
print(labels.shape)


class VariationalAutoEncoder(nn.Module):
      def __init__(self):
          super(VariationalAutoEncoder, self).__init__()
          self.fc1 = nn.Linear(784, 400)
          self.fc21 = nn.Linear(400, 20)
          self.fc22 = nn.Linear(400, 20)
          self.fc3 = nn.Linear(20, 400)
          self.fc4 = nn.Linear(400, 784)
      def encode_function(self, x):
          h1 = F.relu(self.fc1(x))
          return self.fc21(h1), self.fc22(h1)
      def reparametrize(self, mu, logvar):
          std = logvar.mul(0.5).exp_()
          if torch.cuda.is_available():
             eps = torch.cuda.FloatTensor(std.size()).normal_()
          else:
              eps = torch.FloatTensor(std.size()).normal_()

          eps = Variable(eps)
          return eps.mul(std).add_(mu)
      def decode_function(self, z):
          h3 = F.relu(self.fc3(z))
          return torch.sigmoid(self.fc4(h3))
      def forward(self, x):
          mu, logvar = self.encode_function(x)
          z = self.reparametrize(mu, logvar)
          return self.decode_function(z), mu, logvar

model = VariationalAutoEncoder()

from torchsummary import summary
summary(model, input_size=(1, 28* 28)) 

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
weight_decay=1e-5)





def loss_function(reconstruction_x, x, mu, latent_log_variance):
     #BCE = reconstruction_function(reconstruction_x, x)
     BCE = criterion(reconstruction_x, x)
# KL loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
     KLD_aspect =mu.pow(2).add_(latent_log_variance.exp()).mul_(-1).add_(1).add_(logvar)
     KLD = torch.sum(KLD_aspect).mul_(-0.5)
     return BCE + KLD


def to_image(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


for epoch in range(number_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(data_loader):
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        if torch.cuda.is_available():
           img = img.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
           print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,
                batch_idx * len(img),len(data_loader.dataset), 100. * batch_idx /len(data_loader),
                train_loss / len(img)))
           print('Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss /
                  len(data_loader.dataset)))
        if epoch % 10 == 0:
           save = to_image(recon_batch.cpu().data)
           save_image(save, 'E:/Amrita_2024/DEEP LEARNING FOR SIGNAL & IMAGE PROCESSING/Python_neural network/image_{}.png'.format(epoch))
torch.save(model.state_dict(), './sim_autoencoder.pth')

plt.imshow(save[save.shape[0]-1].numpy().squeeze(), cmap='gray_r');
